# Experiments

## Preliminary

In the following, let $G=(V, E)$ be a graph, $P$ denotes the transition probability matrix for each Markov chain, $\pi$ denotes the stationary distribution with respect to $P$, and $\sigma, \tau, \eta\in\{-1, +1\}^V$.

- Hamiltonian on the spin configuration space $\{\pm 1\}^{V}$
$$
    H(\sigma)
    = -\sum_{\{x, y\}\in E}J_{x, y} \sigma_x \sigma_y - \sum_{x\in V} h_x \sigma_x
    = -\frac{1}{2}\sum_{x, y\in V}J_{x, y} \sigma_x \sigma_y - \sum_{x\in V} h_x \sigma_x
$$
- Hamiltonian on $(\{\pm 1\}^{V^\mathrm{L}}, \{\pm 1\}^{V^\mathrm{R}})$ of the bipartite graph $(G^\mathrm{L}, G^\mathrm{R})$
$$
    \tilde H(\tau, \sigma) = -\frac{1}{2}\sum_{x, y\in V}J_{x, y} \tau_x \sigma_y - \frac{1}{2}\sum_{x\in V} h_x \left(\tau_x + \sigma_x\right) + \frac{1}{2}\sum_{x\in V}q_x \left( 1 - \tau_x \sigma_x\right)
$$
- Cavity field
$$
    \tilde h_x(\sigma) = \sum_{y\in V}J_{x, y} \sigma_y + h_x
$$

### Importing libraris

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
#import sys
#sys.path.append('../python')
#import simulator
import simulatorWithCpp as simulator
import math

%matplotlib inline
np.set_printoptions(threshold=16, edgeitems=8)

### Constant numbers (parameters)

We fix the parameters:

- the number of nodes $= 256$,
- the maximum monte carlo steps $= 1000$,
- the number of trials of an annealing $= 200$,
- the initial spin configuration given uniformly at random by either PCG64 in the Python library or MT19937_64 in the C++ library with the random seed $1024$.

In [ ]:
MaxSteps = int(1.e3)
MaxTrials = int(2.e2)
NumNodes = 256
SeedForConfiguration = 1024

### Subroutines to sample and draw a graph

Note that `Energy` property means the original Hamiltonian for the Metropolis method or the Glauber dynamics, whereas it means the Hamiltonian on the bipartite graph for the SCA, the MA or the MMA.

In [ ]:
isingModel = simulator.IsingModel({node: 0.e0 for node in range(NumNodes)}, {})
isingModel.SetSeed(SeedForConfiguration)
isingModel.GiveSpins(simulator.ConfigurationsType.Uniform)
InitialConfiguration = isingModel.Spins

def TryExperimentFor(isingModel, initialTemperature):
    finalEnergies = np.empty(0, dtype=np.float)
    samples = np.empty((0, 3), dtype=np.float)
    for i in range(MaxTrials):
        isingModel.Spins = InitialConfiguration
        isingModel.SetSeed()
        if i == 0:
            isingModel.Temperature = initialTemperature
            isingModel.Write()
            print()
        for n in range(MaxSteps + 1):
            isingModel.Temperature = initialTemperature * 0.99 ** n
            #isingModel.Temperature = 100 * np.exp(-0.005 * n)
            isingModel.Update()
            if i == MaxTrials - 1:
                samples = np.append(samples, np.array([n, isingModel.Energy, isingModel.Temperature], dtype=np.float).reshape((1, 3)), axis=0)
        finalEnergies = np.append(finalEnergies, isingModel.Energy)
        
    print('Mean: {}'.format(np.mean(finalEnergies)))
    print('Standard deviation: {}'.format(np.std(finalEnergies)))
    print('Mode: {}'.format(stats.mode(finalEnergies)))
    print('Minimum: {}'.format(np.min(finalEnergies)))

    fig = plt.figure(figsize=(7, 3), dpi=200)
    ax = fig.add_subplot(121, xlabel='step', ylabel='Energy')
    ax.grid()
    ax.plot(samples[:, 0], samples[:, 1])
    ax = fig.add_subplot(122, xlabel='Energy', ylabel='count')
    ax.grid(which='both')
    ax.hist(finalEnergies, bins=30)
    plt.subplots_adjust(wspace=0.3)
    plt.show()

## Algorithms

### Metropolis method

- Stationary distribution
$$
    \pi(\sigma) = \frac{\exp\bigl(-\beta H(\sigma)\bigr)}{\sum_{\sigma'}\exp\bigl(-\beta H(\sigma')\bigr)}.
$$
- Transition probability
$$
    P(\tau \mid \sigma) = \begin{cases}
        \displaystyle \frac{1}{\lvert V\rvert} \min\left\{1, \exp\bigl(-\beta \Delta E\bigr)\right\} & [\tau = \sigma^x],\\
        \displaystyle 1 - \sum_{x\in V} P_{\sigma, \sigma^x} & [\tau = \sigma],\\
        0 & [\text{otherwise}],
    \end{cases}
$$
where $\Delta E \equiv H(\sigma^x) - H(\sigma) = 2 \tilde h_x(\sigma) \sigma_x$.
- Then, the detailed balance condition
$$
    \pi(\sigma) P(\tau \mid \sigma) = \pi(\tau) P(\sigma \mid \tau)
$$
is satisfied.

### Glauber dynamics

- Stationary distribution
$$
    \pi(\sigma) = \frac{\exp\bigl(-\beta H(\sigma)\bigr)}{\sum_{\sigma'}\exp\bigl(-\beta H(\sigma')\bigr)}.
$$
- Transition probability
$$
    P(\tau \mid \sigma) = \begin{cases}
        \displaystyle \frac{1}{\lvert V\rvert}\frac{\exp\bigl(-\beta \tilde h_x(\sigma) \sigma_x\bigr)}{2 \cosh\bigl(\beta \tilde h_x(\sigma)\bigr)} & [\tau = \sigma^x],\\
        \displaystyle 1 - \sum_{x\in V} P_{\sigma, \sigma^x} & [\tau = \sigma],\\
        0 & [\text{otherwise}].
    \end{cases}
$$
- Then, the detailed balance condition
$$
    \pi(\sigma) P(\tau \mid \sigma) = \pi(\tau) P(\sigma \mid \tau)
$$
is satisfied.

### Stochastic cellular automata

- Stationary distribution on the bipartite graph $(G^\mathrm{L}, G^\mathrm{R})$
$$
    \pi(\tau, \sigma) = \frac{\exp\bigl(-\beta\tilde H(\tau, \sigma)\bigr)}{\sum_{\tau', \sigma'}\exp\bigl(-\beta\tilde H(\tau', \sigma')\bigr)}.
$$
- Transition probability (Glauber dynamics on the bipartite graph $(G^\mathrm{L}, G^\mathrm{R})$)
\begin{align*}
    P(\tau, \sigma \mid \eta, \sigma) &= \frac{\exp\bigl(-\beta\tilde H(\tau, \sigma)\bigr)}{\sum_{\tau'}\exp\bigl(-\beta\tilde H(\tau', \sigma)\bigr)}\\
    &= \prod_{x\in V} \frac{\exp\bigl(-\beta (\tilde h_x(\sigma) + q_x\sigma_x) \tau_x / 2\bigr)}{2\cosh\bigl(\beta (\tilde h_x(\sigma) + q_x\sigma_x) / 2\bigr)}\\
    &= \prod_{x\in V} \frac{1}{1 + \exp\bigl(-\beta (\tilde h_x(\sigma) + q_x\sigma_x) \tau_x\bigr)}.
\end{align*}
- Then, the detailed balance condition
$$
    \pi(\eta, \sigma) P(\tau, \sigma \mid \eta, \sigma) = \pi(\tau, \sigma) P(\eta, \sigma \mid \tau, \sigma)
$$
is satisfied.
- Let $L_x\sim\mathrm{Logistic}(0, 1)$ for each $x\in V$ and $T = 1 / \beta$.  We regard $\sigma$ as the current spin configuration and $\tau$ as the next spin configuration in the MCMC updating.  Then, the algorithm is represented by
$$
    \DeclareMathOperator{\sgn}{sgn}
    \tau_x = \sgn\left(\tilde h_x(\sigma) + q_x\sigma_x - TL_x\right).
$$

### Momentum annealing

(In this section, we do not control temperature, so that this MCMC algorithm is not "annealing".  However, since the MCMC do not have any name, we call it the "momentum annealing" for convinience)

- Stationary distribution on the bipartite graph $(G^\mathrm{L}, G^\mathrm{R})$
$$
    \pi(\tau, \sigma) = \frac{\exp\bigl(-\beta\tilde H(\tau, \sigma)\bigr)}{\sum_{\tau', \sigma'}\exp\bigl(-\beta\tilde H(\tau', \sigma')\bigr)}.
$$
- When we update the spins on $V^\mathrm{L}$, the energy difference is expressed by
\begin{align*}
    \Delta E &\equiv \tilde H(\tau, \sigma) - \tilde H(\eta, \sigma)\\
    &= \frac{1}{2} \sum_{x\in V} \left(\tilde h_x(\sigma) + q_x \sigma_x\right) \left(\eta_x - \tau_x\right)\\
    &= \sum_{\substack{x\in V\\ (\tau_x = -\eta_x)}} \left(\tilde h_x(\sigma) + q_x \sigma_x\right) \eta_x
\end{align*}
for $\eta, \tau\in\{-1, +1\}^{V^\mathrm{L}}$ and $\sigma\in\{-1, +1\}^{V^\mathrm{R}}$.  Let $\Delta E_x = \left(\tilde h_x(\sigma) + q_x \sigma_x\right) \eta_x$.
- Transition probability (Metropolis method on the bipartite graph $(G^\mathrm{L}, G^\mathrm{R})$)
\begin{align*}
    P(\tau, \sigma \mid \eta, \sigma)
    &= \prod_{x\in V} \left(\min\left\{1, \mathrm{e}^{-\beta \Delta E_x}\right\} \delta_{\tau_x, -\eta_x} + \left( 1 - \min\left\{1, \mathrm{e}^{-\beta \Delta E_x}\right\}\right) \delta_{\tau_x, \eta_x}\right)\\
    &= \left(\prod_{\substack{x\in V\\ (\tau_x = -\eta_x)}} \min\left\{1, \mathrm{e}^{-\beta \Delta E_x}\right\}\right)
        \left(\prod_{\substack{y\in V\\ (\tau_y = \eta_y)}} \left( 1 - \min\left\{1, \mathrm{e}^{-\beta \Delta E_y}\right\}\right)\right).
\end{align*}
- Then, the detailed balance condition
$$
    \pi(\eta, \sigma) P(\tau, \sigma \mid \eta, \sigma) = \pi(\tau, \sigma) P(\eta, \sigma \mid \tau, \sigma)
$$
is satisfied.
- Let $\varGamma_x\sim\mathrm{Exponential}(1)$ for each $x\in V$ and $T = 1 / \beta$.  We regard $\sigma$ as the current spin configuration, $\tau$ as the next spin configuration and $\eta$ as the previous spin configuration in the MCMC updating.  Then, the algorithm is represented by
$$
    \DeclareMathOperator{\sgn}{sgn}
    \tau_x = \sgn\left(\tilde h_x(\sigma) + q_x\sigma_x - T\varGamma_x\eta_x\right).
$$

### Modified momentum annealing

- Let $\varGamma_x\sim\mathrm{Exponential}(1)$ for each $x\in V$ and $T = 1 / \beta$.  We regard $\sigma$ as the current spin configuration and $\tau$ as the next spin configuration in the MCMC updating.  Then, the algorithm is represented by
$$
    \DeclareMathOperator{\sgn}{sgn}
    \tau_x = \sgn\left(\tilde h_x(\sigma) + q_x\sigma_x - T\varGamma_x\sigma_x\right).
$$

### Annealing schedule

For $n=\mathbb{N}\cup\{0\}$,
$$
    T_n = T_0 \cdot 0.99^n.
    %T_n = 100 \cdot \mathrm{e}^{-0.005 n}.
$$

## Square lattice

Our experiment runs under the conditions:

- no external magnetic field,
- antiferromagnet,
- the free boundary condition.

In [ ]:
def GenerateSquareLatticeEdges(numNodes):
    columns = math.ceil(math.sqrt(numNodes))
    result = {}
    for i in range(numNodes - 1):
        if (i + 1) % columns > 0:
            result[(i, i + 1)] = -1
        if (i + columns) < numNodes:
            result[(i, i + columns)] = -1
    return result

In [ ]:
# Test for the function GenerateSquareLatticeEdges.
for i in range(16):
    if i != 0 and i % 4 == 0:
        print()
    print('{:3d}'.format(i), end='')
print('\n')
quadratic = GenerateSquareLatticeEdges(16)
print(quadratic)
print('\n')
isingModel = simulator.IsingModel({}, quadratic)
isingModel.Write()

In [ ]:
# Actual simulation.
quadratic = GenerateSquareLatticeEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

### Metropolis method

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Metropolis
TryExperimentFor(isingModel, T0)

### Glauber dynamics

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Glauber
TryExperimentFor(isingModel, T0)

### Stochastic cellular automata

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Modified momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MMA
isingModel.PinningParameter = isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

## Complete graph

Our experiment runs under the conditions:

- no external magnetic field,
- antiferromagnet.

In [ ]:
def GenerateCompleteGraphEdges(numNodes):
    return {(i, j): -1 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateCompleteGraphEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

### Metropolis method

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Metropolis
TryExperimentFor(isingModel, T0)

### Glauber dynamics

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Glauber
TryExperimentFor(isingModel, T0)

### Stochastic cellular automata

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Modified momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MMA
isingModel.PinningParameter = isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

## Erdős-Rényi random graph

Our experiment runs under the conditions:

- no external magnetic field,
- antiferromagnet,
- the occupation probability $=0.5$,
- Bernoulli random numbers given by MT19937 with the random seed $2048$.

In [ ]:
OccupationProbability = 0.5e0
SeedForRandomGraph = 2048

def GenerateErdosRenyiEdges(numNodes, probability):
    rng = np.random.Generator(np.random.MT19937(SeedForRandomGraph))
    return {(i, j): -1 if rng.random() <= probability else 0 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateErdosRenyiEdges(NumNodes, OccupationProbability)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

### Metropolis method

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Metropolis
TryExperimentFor(isingModel, T0)

### Glauber dynamics

In [ ]:
isingModel.Algorithm = simulator.Algorithms.Glauber
TryExperimentFor(isingModel, T0)

### Stochastic cellular automata

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

### Modified momentum annealing

In [ ]:
isingModel.Algorithm = simulator.Algorithms.MMA
isingModel.PinningParameter = isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)